In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import redditutils as ru

from gensim.models import Word2Vec
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
df = pd.read_csv('showerthoughts.csv')

In [11]:
df['title'][:100].apply(lambda x: ru.strip_non_eng(x))

0     Everyone that ' s ever said " I ' m speechless...
1     I wonder how many people I ' to who have been ...
2     " Palindrome " is a let down as a word . Rhino...
3     When you tell someone to keep a secret , it is...
4     Someone had the the time to create the , " Dic...
5     are basically baby of fruit and now I feel hor...
6     Nothing ever like the length of time it ' s su...
7     If there were before we , we probably would ha...
8         Life is like , on you ' re exploring deep web
9     If the Star universe had an and he would have ...
10    At some point , the top post of / r / was prob...
11    If they ever make a device that ' t blink or m...
12    With the avalanche , probably more people than...
13    Which whale and dolphin species are worth flyi...
14    I wonder how many people scale down " Secret "...
15                       If you ask to be you cannot be
16    There should be a word for the phenomenon of a...
17        Rising and Ascending are basically syn

In [25]:
df['title'] = df['title'].apply(lambda x: ru.clean_title(x))

In [28]:
df['title'] = df['title'].apply(lambda x: ru.strip_non_eng(x))

In [37]:
df['title'] = df['title'].apply(lambda x: ru.strip_stops(x))

In [40]:
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')

In [41]:
df.head()

,author,title,subreddit,score,num_comments,retrieved_on,id,created_utc,selftext
0,os_coxae,everyone thats ever said speechless liar,Showerthoughts,0,0,1466445837,4ixl2n,2016-05-11 22:38:48,NaN
1,Calrizle,wonder many people conversation,Showerthoughts,0,0,1466494743,4lfjxn,2016-05-28 12:03:56,NaN
2,[deleted],palindrome let word rhinoplasty sure,Showerthoughts,0,0,1466493929,4ldt85,2016-05-28 01:21:03,[deleted]
3,[deleted],tell someone keep secret longer secret,Showerthoughts,0,0,1466423373,4hmcbg,2016-05-03 08:30:35,[deleted]
4,[deleted],someone time create dick penis vagina,Showerthoughts,0,0,1466474842,4k9ngz,2016-05-20 18:29:28,[deleted]


In [42]:
df = df[['title', 'score', 'id', 'created_utc', 'selftext', 'num_comments']]

In [ ]:
df.to_csv('cleaned_shower.csv')